# Palmer Amaranth (Amaranthus palmeri) Growth Stage Detection
This Google Colab notebook has been adapted from the **official YOLOv3 notebook** by **Ultralytics** to train and evaluate YOLOv3 on the Palmer amaranth Growth Stage (PAGS8) dataset. The dataset is available for download from [Weed-AI](). Prior to use, the dataset should be setup using the instructions from the official repository.

It accompanies the preprint: **Multi-growth stage plant recognition: a case study of Palmer amaranth (Amaranthus palmeri) in cotton (Gossypium hirsutum)** available on [arXiv](https://arxiv.org/abs/2307.15816). Please consider citing this preprint if you use the work in your research.

Models were trained using Google Colab Pro+ with access to an NVIDIA A100-SXM4-40GB.

This notbook consists of three parts:
1. Setup
2. Training
3. Evaluation

For additional documentation, training options and inference procedures please visit the official Ultralytics repository. (Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)

<img src="https://github.com/geezacoleman/Palmer-detection/assets/51358498/6040227e-a072-43bf-a789-72e0833f3168"/>

# 1. Setup

In [ ]:
# mount google drive
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
# Assumes you have already cloned the Palmer-detection repository
%cd '/content/drive/MyDrive/Palmer-detection'

In [ ]:
%cd yolov5
!pip install -r requirements.txt

In [ ]:
# import the necessary packages
import torch
import os 
import random

from datetime import datetime

random.seed(0)
DATE = datetime.now().strftime('%Y%m%d')
IMAGE_SIZE = 1280

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

# 2. Training 

### Weights and Biases
WandB is an excellent tool to track/monitor training performance, particularly across large training runs. While optional, it is recommended, however, requires setting up an account and logging in below.

Only run this cell if you plan on using WandB.

In [ ]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

Check the dataset has been set up correctly

In [ ]:
%cd '/content/drive/MyDrive/Palmer-detection/datasets/fold_0/8cls_fold_0'
%ls
print(len(os.listdir('images/train')), len(os.listdir('labels/train')))
print(len(os.listdir('images/val')), len(os.listdir('labels/val')))
print(len(os.listdir('images/test')), len(os.listdir('labels/test')))

## Class Grouping

In [ ]:
%cd '/content/drive/My Drive/Palmer-detection/yolov5'
class_groups = ['8', '8-sz-', '3', '3-sz-', '1'] # remove 1280 if you have already trained other versions

# p5 MODELS 8 classes - different folds
for group in class_groups:
  for fold in range(0, 5):
    yaml_path = f"{group}cls_fold_{fold}.yaml"
    full_name = f'{DATE}_yolov5m5_1280_B8_F{fold}_{group}cls_default'
    print(full_name)
    !python train.py --img {IMAGE_SIZE} --cfg yolov5m.yaml --hyp hyp.scratch-low.yaml --batch 8 --epochs 30 --data {yaml_path} --weights yolov5m.pt --name {full_name}


## Image Size
Train an all eight classes for four different resolutions, three model sizes and five folds.

In [ ]:
%cd '/content/drive/My Drive/Palmer-detection/yolov5'
image_sizes = [1600, 1280, 320, 640] # remove 1280 if you have already trained other versions
models = ['n', 'm', 'x']

# p5 MODELS 8 classes - different folds
for image_size in image_sizes:
  for model in models:
    for fold in range(0, 5):
      yaml_path = f"8cls_fold_{fold}.yaml"
      full_name = f'{DATE}_yolov5{model}5_{image_size}_B8_F{fold}_8cls'
      print(full_name)
      !python train.py --img {image_size} --cfg yolov5{model}.yaml --hyp hyp.scratch-low.yaml --batch 8 --epochs 30 --data {yaml_path} --weights yolov5{model}.pt --name {full_name}


## Model Variants
Train all YOLOv5 model variants.

In [ ]:
# FOLDS
%cd '/content/drive/My Drive/Palmer-detection/yolov5'
models = ['x6', 'n6', 'm6', 'n', 's', 'm', 'l', 'x'] # this will train all model variants - remove as necessary.
classes = ['1', '8']

# p5 MODELS 8 classes - different folds
for model in models:
  for class_num in classes:
    for fold in range(0, 5):
      yaml_path = f"{class_num}cls_fold_{fold}.yaml"
      full_name = f'{DATE}_yolov5{model}_B8_F{fold}_{class_num}cls'
      print(full_name)
      
      !python train.py --img 1280 --cfg yolov5{model}.yaml --hyp hyp.scratch-low.yaml --batch 8 --epochs 30 --data {yaml_path} --weights yolov5{model}.pt --name {full_name}

# 3. Evaluation
Run val.py on all the training runs

In [ ]:
train_path = "/content/drive/My Drive/Palmer-detection/yolov5/runs/train"
%cd '/content/drive/My Drive/Palmer-detection/yolov5'

for run_name in os.listdir(train_path):
  if 'v5' not in run_name:
    print(f'Skipping {run_name}')
  else:
    fold = run_name.split("_")[-3][-1]
    group = run_name.split("_")[-2][:-3]
    
    yaml_path = f"{class_num}cls_fold_{fold}.yaml"
    
    !python val.py --img 1280 --weights runs/train/{run_name}/weights/best.pt --data data/{yaml_path} --name {run_name} --task 'test' --batch-size 8 --iou-thres 0.6 --conf-thres 0.001 --verbose --save-metrics
